In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
import flopy

# SWF Model does not yet have the conveyance formulation
# required to run this example

In [ ]:
# problem described here: https://www.hec.usace.army.mil/confluence/hmsdocs/hmsguides/applying-reach-routing-methods-within-hec-hms/applying-the-muskingum-cunge-routing-method
# These are the hec-hms results for Punxsutawney for a single
# reach solved using Muskingum-Cunge
import punx_data
data = punx_data.get_data()
punx_inflow_hydrograph = data["inflow_hydrograph"]
punx_hec_hms_outflow = data["hec_hms_outflow"]
punx_obs_outflow = data["obs_outflow"]
sample_times = data["sample_times"]
dt = 15 * 60 # 15 mins converted to seconds
total_time = punx_inflow_hydrograph.shape[0] * dt
print(f"{total_time=}")

In [ ]:
sec_to_min = 60
plt.plot(sample_times / sec_to_min, punx_inflow_hydrograph, 'go', label="inflow")
plt.plot(sample_times / sec_to_min, punx_hec_hms_outflow, 'b-', label="HEC-HMS outflow")
plt.plot(sample_times / sec_to_min, punx_obs_outflow, 'bo', label="observed outflow")
plt.xlabel("time, in minutes")
plt.ylabel("flow, in cms")
plt.legend()
plt.title("HEC-HMS Punxsatawney Example")

In [ ]:
txt = "Punxsutawney Cross Section"
# select the full cross section or the 8 point
# section_name = "full"
section_name = "8point"
channel_n = 0.040 #0.035
bank_n = 0.09 #0.15
cross_section_data = punx_data.get_cross_section_data(
    section_name=section_name,
    channel_n=channel_n,
    bank_n=bank_n,
)

# override with this rectangular cross section
# cross_section_data["x"] = np.array([0., 0., 20., 20.])
# cross_section_data["h"] = np.array([10., 0., 0., 10.])
# cross_section_data["r"] = np.array([channel_n, channel_n, channel_n, 0.])

# trapezoidal-like cross section
# cross_section_data["x"] = np.array([
#     0.,
#     37.39941478,
#     41.09973177,
#     61.39965862,
#     68.6021702,
#     105.19995123
# ])
# cross_section_data["h"] = np.array([
#     10.70013411,
#     4.60009754,
#     0.,
#     0.6000061,
#     4.60009754,
#     10.70013411
# ])
# cross_section_data["r"] = [bank_n, channel_n, channel_n, channel_n, bank_n, 0.]

# This data has metric units
x = cross_section_data["x"]
h = cross_section_data["h"]
r = cross_section_data["r"]

from matplotlib import cm
plt.plot(x, h, 'bo')
for i in range(x.shape[0] - 1):
    plt.plot(x[i:i+2], h[i:i+2], lw=r[i]*20, color="blue", label=f"rough={r[i]}")
plt.xlabel("station, in m")
plt.ylabel("elevation, in m")
plt.title(txt)
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())


In [ ]:
x, h, r

In [ ]:
stream_length = 68861. / 3.2808 # converted to  meters
total_time = sample_times[-1]
nreach = 8 * 1
nstp = 72 * 1
dx = stream_length / nreach
dt = total_time / nstp
celerity = 5. / 3.2808 # convert 5 feet per second to meters per second
print(f"{stream_length=:0.2f} meters")
print(f"{total_time=} seconds")
print(f"{nreach=}")
print(f"{nstp=}")
print(f"{dx=:0.2f} meters")
print(f"{dt=} seconds")
print(f"desired dx based on courant = {celerity * dt} meters")
print(f"desired dt based on courant = {dx / celerity} seconds")

In [ ]:
# set up reach bottom
dz = 82.0 / 3.2808 # elevation change from upstream to downstream
slope = dz / stream_length
print(f"{dz=}")
print(f"{slope=}")
x = np.linspace(dx / 2, stream_length - dx / 2, nreach)
reach_bottom = x * slope
reach_bottom = reach_bottom[::-1]

# set up vertices
vertices = []
vertices = [[j, j * dx, 0.0, 0.0] for j in range(nreach + 1)]
cell2d = []
for j in range(nreach):
    cell2d.append([j, 0.5, 2, j, j + 1])


In [ ]:
# make modflow model
exe = "/Users/langevin/langevin/dev/modflow6-fork.git/bin/mf6"
sim_ws = Path("./mf6_punx")
name = "punx"
sim = flopy.mf6.MFSimulation(
    sim_name=name, 
    version="mf6", 
    exe_name=exe, 
    sim_ws=sim_ws,
    memory_print_option='summary',
    continue_=False,
)
tdis = flopy.mf6.ModflowTdis(sim, 
    nper=1, 
    perioddata=[(total_time, nstp, 1.0)], 
    time_units="seconds"
)

if False:
    # set dt0, dtmin, dtmax, dtadj, dtfailadj
    dt0 = 60 * 1.0  # 1 min (in seconds)
    dtmin = 1.0  # (in seconds)
    dtmax = 60 * 30.  # 30 min (in seconds)
    dtadj = 2.0
    dtfailadj = 5.0
    ats_filerecord = name + ".ats"
    atsperiod = [
        (0, dt0, dtmin, dtmax, dtadj, dtfailadj),
    ]
    tdis.ats.initialize(
        maxats=len(atsperiod),
        perioddata=atsperiod,
        filename=ats_filerecord,
    )


dvclose = 1.e-8
ims = flopy.mf6.ModflowIms(
    sim,
    print_option="summary",
    outer_dvclose=dvclose,
    outer_maximum=100,
    under_relaxation="DBD",
    under_relaxation_theta=0.95,
    under_relaxation_kappa=0.0001,
    under_relaxation_gamma=0.0,
    under_relaxation_momentum=0.0,
    inner_maximum=20,
    inner_dvclose=dvclose,
    linear_acceleration="BICGSTAB",
    scaling_method="NONE",
    reordering_method="NONE",
    relaxation_factor=0.97,
    # backtracking_number=5,
    # backtracking_tolerance=1.0,
    # backtracking_reduction_factor=0.3,
    # backtracking_residual_limit=100.0,
    csv_outer_output_filerecord=f"{name}.ims.outer.csv",
)

swf = flopy.mf6.ModflowSwf(
    sim, 
    modelname=name, 
    save_flows=True,
    newtonoptions=True,
)
disl = flopy.mf6.ModflowSwfdisl(
    swf, 
    nodes=nreach, 
    reach_length=dx,
    reach_bottom=reach_bottom,
    reach_width=1.,
    vertices=vertices,
    cell2d=cell2d,
    idomain=1, 
)
ic = flopy.mf6.ModflowSwfic(swf, strt=reach_bottom + 1.0)
slope = 0.0012
dfw = flopy.mf6.ModflowSwfdfw(
    swf,
    print_flows=True,
    save_flows=True,
    manningsn=1.,
    idcxs=0,
)
sto = flopy.mf6.ModflowSwfsto(
    swf,
    save_flows=True,
)
xfraction = cross_section_data["x"]
height = cross_section_data["h"]
mannfraction = cross_section_data["r"]
npoints = xfraction.shape[0]
cxsdata = list(zip(xfraction, height, mannfraction))
cxs = flopy.mf6.ModflowSwfcxs(
    swf,
    nsections=1,
    npoints=npoints,
    packagedata=[(0, npoints)],
    crosssectiondata=cxsdata,
)
flw = flopy.mf6.ModflowSwfflw(
    swf,
    maxbound=1,
    print_input=True,
    print_flows=True,
    stress_period_data=[(0, "inflow")],
)
fname = f"{name}.flw.ts"
flw.ts.initialize(
    filename=fname,
    timeseries=list(zip(sample_times, punx_inflow_hydrograph)),
    time_series_namerecord=["inflow"],
    interpolation_methodrecord=["linearend"],
)

fname = f"{name}.zdg.obs.csv"
zdg_obs = {
    fname: [
        ("OUTFLOW", "ZDG", (nreach - 1,)),
    ],
    "digits": 10,
}
idcxs = 0  # use cross section 0
width = 1.0
slope = slope
rough = 1.0
spd = [((nreach - 1,), idcxs, width, slope, rough)]
zdg = flopy.mf6.ModflowSwfzdg(
    swf,
    observations=zdg_obs,
    print_input=True,
    maxbound=len(spd),
    stress_period_data=spd,
)

oc = flopy.mf6.ModflowSwfoc(
    swf,
    budget_filerecord=f"{name}.bud",
    stage_filerecord=f"{name}.stage",
    saverecord=[
        ("STAGE", "ALL"),
        ("BUDGET", "ALL"),
    ],
    printrecord=[
        ("STAGE", "LAST"),
        ("BUDGET", "ALL"),
    ],
)
obs_data = {
    f"{name}.obs.csv": [
        ("STAGE1", "STAGE", (0,)),
        (f"STAGE{nreach}", "STAGE", (nreach - 1,)),
    ],
}
obs_package = flopy.mf6.ModflowUtlobs(
    swf,
    filename=f"{name}.obs",
    digits=10,
    print_input=True,
    continuous=obs_data,
)

sim.write_simulation()
success, buff = sim.run_simulation(silent=False)
assert success, buff

In [ ]:
fpth = Path(sim_ws) / f"{name}.zdg.obs.csv"
obsvals = np.genfromtxt(fpth, names=True, delimiter=",")
sim_outflow = -obsvals["OUTFLOW"]
sim_times = obsvals["time"]
plt.plot(sample_times, punx_inflow_hydrograph, 'ro-', label="Inflow")
plt.plot(sim_times, sim_outflow, 'b-', label="MODFLOW 6")
plt.plot(sample_times, punx_obs_outflow, marker="o", mec="k", mfc="none", ls="", label="HEC-RAS")
#plt.plot(sample_times, punx_hec_hms_outflow, 'g-', label="HEC-HMS")
plt.legend()
plt.ylabel("flow, in cms")
plt.xlabel("time, in seconds")

In [ ]:
fpth = Path(sim_ws) / f"{name}.obs.csv"
obsvals = np.genfromtxt(fpth, names=True, delimiter=",")
sim_times = obsvals["time"]
sim_stage1 = obsvals["STAGE1"]
sim_stagen = obsvals[f"STAGE{nreach}"]
sim_depth1 = sim_stage1 - reach_bottom[0]
sim_depthn = sim_stagen - reach_bottom[nreach - 1]
plt.plot(sim_times, sim_depth1, 'bo-', label="MODFLOW 6 Stage 1")
plt.plot(sim_times, sim_depthn, 'g-', label=f"MODFLOW 6 Stage {nreach}")
plt.legend()
plt.ylabel("water depth, in meters")
plt.xlabel("time, in seconds")

In [ ]:
def get_x_positions(vertices, cell2d):
    x = []
    for cell_data in cell2d:
        icell_number = cell_data[0]
        vstart = cell_data[-2]
        vend = cell_data[-1]
        x.extend(vertices[[vstart, vend]]["xv"])
    return x

def get_z_positions(zdata, vertices, cell2d):
    z = []
    for cell_data in cell2d:
        icell_number = cell_data[0]
        vstart = cell_data[-2]
        vend = cell_data[-1]
        z.extend(2 * [zdata[icell_number]])
    return z

def plot_channel_bottom(ax, vertices, cell2d, reach_bottom):
    x = get_x_positions(vertices, cell2d)
    z = get_z_positions(reach_bottom, vertices, cell2d)    
    ax.plot(x, z, "k--", lw=1)
    return

def get_patch_bounds(stage, vertices, cell2d, reach_bottom):
    """
    Create information for rectangular patches that span
    from stage to reach_bottom
    
    """
    xy = []
    width = []
    height = []
    for cell_data in cell2d:
        icell_number = cell_data[0]
        vstart = cell_data[-2]
        vend = cell_data[-1]
        xv0 = vertices[vstart]["xv"]
        xv1 = vertices[vend]["xv"]
        ztop = stage[icell_number]
        zbot = reach_bottom[icell_number]
        xy.append((xv0, zbot))
        width.append(xv1 - xv0)
        height.append(max(ztop - zbot, 0.))
    return xy, width, height

def get_patch_list(xy_list, width_list, height_list):
    from matplotlib.patches import Rectangle
    pc_list = []
    for xy, width, height in zip(xy_list, width_list, height_list):
        pc_list.append(Rectangle(xy, width, height))
    return pc_list

In [ ]:
fig, ax = plt.subplots(figsize=(8, 3))
ax.set_xlabel(r'x')
ax.set_ylabel(r'z')
ax.set_ylim(0, 35)
title = ax.set_title(f"{0} percent completed")

# plot persistent items
vertices = swf.disl.vertices.get_data()
cell2d = swf.disl.cell2d.get_data()
reach_bottom = swf.disl.reach_bottom.get_data()
plot_channel_bottom(ax, vertices, cell2d, reach_bottom)

patch_data = get_patch_bounds(reach_bottom, vertices, cell2d, reach_bottom)
pc_list = get_patch_list(*patch_data)
pc = PatchCollection(pc_list, facecolor="blue", alpha=0.5,
                         edgecolor="none")
ax.add_collection(pc)

fpth = sim_ws / f"{swf.name}.stage"
sobj = flopy.utils.HeadFile(fpth, precision="double", text="STAGE")
times = sobj.get_times()
stage_all = sobj.get_alldata().squeeze()
    
def animate(i):
    stage = stage_all[i]
    
    patch_data = get_patch_bounds(stage, vertices, cell2d, reach_bottom)
    pc_list = get_patch_list(*patch_data)
    
    pc.set_paths(pc_list)
    title = ax.set_title(f"{times[i] / times[-1] * 100.:0.2f} percent completed")
    return

import matplotlib.animation
ani = matplotlib.animation.FuncAnimation(fig, animate, frames=stage_all.shape[0])

plt.close()
from IPython.display import HTML
HTML(ani.to_jshtml())